# Analysis of the data collected by GCOZ

In [ ]:
import json
import pandas as pd
import seaborn as sb
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def iqr_averages(df : pd.DataFrame):
    averages = []
    for col in df:
        q1 = df[col].quantile(.25)
        q3 = df[col].quantile(.75)
        iqr = 1.3 * (q3 - q1)
        filter = (df[col] >= q1 - 1.5*iqr) & (df[col] <= q3 + 1.5*iqr)
        averages.append(df[col].loc[filter].mean())
    return averages

def reg_averages(df : pd.DataFrame):
    averages = []
    for col in df:
        averages.append(df[col].mean())
    return averages

In [ ]:
_GAME = 'Dirt Rally' # replace with the string you gave the profiler on execution
_RUN  = '0' # replace with the number generated by the profiler on saving
_GRAPHS = f'graphs/{_GAME}/run_{_RUN}'

In [ ]:
with open(f'data/{_GAME}/run_{_RUN}/Frametimes.json') as infile:
    times = json.load(infile)

with open(f'data/{_GAME}/run_{_RUN}/MethodDurations.json') as infile:
    durations =json.load(infile)

with open(f'data/{_GAME}/run_{_RUN}/ThreadIDs.json') as infile:
    ids = json.load(infile)

In [ ]:
# Create Directory for resulting graphs
from pathlib import Path
Path(_GRAPHS).mkdir(parents=True, exist_ok=True)

## Remove uninteresting methods from data

In [ ]:
# get the average Durations above 1000ns, as only these methods get delayed in profiling
averageDurations = {}
for key, data in durations.items():
    avg = np.mean(data)
    if(avg > 1000 or key == 'End'):
        averageDurations[key] = avg
print(json.dumps(averageDurations, indent=4))

In [ ]:
temp = {}

for key, data in times.items():
    if(key in averageDurations or key == "All"):
        print(key)
        temp[key] = pd.DataFrame(data).div(1e6)
times = temp

In [ ]:
all = pd.DataFrame(times["All"])
all.describe()

## Inspect Frametime Data

In [ ]:
rowN, colN = (int(np.ceil(len(times.keys())/3)), 3)
fig, axs = plt.subplots(ncols = colN, nrows = rowN, figsize = (30, 20))
fig.suptitle(f"Resulting frametime by delaying method in {_GAME}")
fig.subplots_adjust(top=.95)

row, col = 0, 0
for key in times.keys():
    times[key].plot(ax=axs[row, col])
    axs[row, col].set_title(key)
    axs[row, col].legend(loc='upper left', ncols = 2)
    axs[row, col].grid()
    axs[row, col].set_ylabel('Time in miliseconds')
    col += 1
    if(col >= colN):
        col = 0
        row += 1

plt.subplots_adjust(wspace=.1)
plt.savefig(f'{_GRAPHS}/AllFrametimesLine.png', bbox_inches = 'tight')

In [ ]:
rowN, colN = (int(np.ceil(len(times.keys())/3)), 3)
fig, axs = plt.subplots(ncols = colN, nrows = rowN, figsize = (30, 10))
fig.suptitle(f"Resulting frametime by delaying method in {_GAME}")

row, col = 0, 0
for key in times.keys():
    sb.boxplot(times[key], showfliers=False , ax=axs[row, col])
    axs[row, col].set_title(key)
    axs[row, col].grid()
    axs[row, col].set_ylabel('Time in miliseconds')
    axs[row, col].set_xlabel('Delay in percentage of original')
    col += 1
    if(col >= colN):
        col = 0
        row += 1

plt.savefig(f'{_GRAPHS}/AllFrametimesBox.png', bbox_inches = 'tight')
plt.subplots_adjust(hspace=.25, wspace=.07, top=1.5)

sfig, saxs = plt.subplots(1, 1, figsize = (20, 5))
sb.boxplot(times["All"], showfliers=False , ax=saxs)
saxs.set_title(f'All Methods Delayed in {_GAME}')
saxs.grid()
saxs.set_ylabel('Time in miliseconds')
saxs.set_xlabel('Delay in percentage of original')
plt.savefig(f'{_GRAPHS}/SingleFrametimesBox.png', bbox_inches = 'tight')

## Average the frametime data

In [ ]:
averageFrametime = pd.DataFrame()
for method, df in times.items():
    averageFrametime[method] = {}
    averageFrametime[method] = iqr_averages(pd.DataFrame(times[method]))
    for delay, data in df.items():
        pass
averageFrametime = averageFrametime.transpose()
averageFrametime.columns = ["0%", "10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%"]

averageFrametime

In [ ]:
fig, axs = plt.subplots(figsize = (25, 10))
averageFrametime.transpose()["All"].plot(ax=axs, kind='bar', ylim=(min(averageFrametime.transpose()["All"] - .1), max(averageFrametime.transpose()["All"] + .1)))
axs.set_title('Average resulting frametime by delaying every method')
axs.set_ylabel('Time(ms)')
axs.set_xlabel('Percentage of original time applied as delay')
plt.xticks(rotation=0, horizontalalignment='center')

plt.savefig(f'{_GRAPHS}/BaselineAverage.png')

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(20,5))
averageFrametime.transpose().plot(ax=axs).legend(loc='upper left', ncols=2)

In [ ]:
allMethods = averageFrametime.transpose()["All"]
singleMethods = averageFrametime.transpose().drop(columns=['All'])

In [ ]:
allMethods

In [ ]:
singleMethods

In [ ]:
differences = np.abs(singleMethods.sub(allMethods, axis='rows'))
differences

In [ ]:
fig, axs  = plt.subplots(1, 1, figsize=(20,5))
axs.set_title('Absolute difference in Frametime')
axs.set_ylabel('Time(ms)')
axs.set_xlabel('Delay applied to all other methods')
differences.plot(ax=axs)
plt.savefig(f'{_GRAPHS}/resultAbs.png')

In [ ]:
results = differences.div(allMethods["0%"], axis=0)
fig, axs = plt.subplots(1, 1, figsize=(20, 5))
axs.grid()
axs.set_title('Relative difference in Frametime')
axs.set_ylabel('Time(ms)')
axs.set_xlabel('Delay applied to all other methods')

results.plot(ax=axs).legend(loc = 'upper left', ncols = 2)
plt.savefig(f'{_GRAPHS}/resultRel.png')